In [2]:
import requests
import pandas as pd
import psycopg2
import os
from dotenv import load_dotenv

# Caminho para o banco DuckDB
DUCKDB_PATH = "populacao_duckdb.db"

# Carrega variáveis do .env
load_dotenv()

# Validação das variáveis obrigatórias
required_vars = ["DB_NAME", "DB_USER", "DB_PASSWORD"]
missing = [var for var in required_vars if os.getenv(var) is None]
if missing:
    raise EnvironmentError(f"Variáveis ausentes no .env: {', '.join(missing)}")

# Configuração do PostgreSQL
PG_CONFIG = {
    "dbname": os.getenv("DB_NAME"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "host": os.getenv("DB_HOST", "localhost"),
    "port": os.getenv("DB_PORT", "5432")
}

# URL da API
URL = "https://servicodados.ibge.gov.br/api/v3/agregados/1209/periodos/2000|2010|2022/variaveis/606?localidades=N3[all]&classificacao=58[all]"

def extrair_dados():
    print("[1/5] Extraindo dados da API IBGE...")
    response = requests.get(URL)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Erro ao acessar API IBGE: {response.status_code}")

def normalize(json_data):
    print("[2/5] Normalizando estrutura JSON...")
    dados = pd.json_normalize(
        json_data,
        record_path=['resultados'],
        meta=['id', 'variavel', 'unidade']
    )
    dados1 = dados.explode('classificacoes').reset_index(drop=True)
    classificacoes_df = pd.json_normalize(dados1['classificacoes'])
    dados2 = dados.explode('series').reset_index(drop=True)
    serie_df = pd.json_normalize(dados2['series'])
    return pd.concat([dados1.drop(columns=['classificacoes']), classificacoes_df,dados2.drop(columns=['series']),serie_df], axis=1)

# Executa funções e exibe os dados
json_data = extrair_dados()
df = normalize(json_data)

print("\n[3/5] Dados extraídos e normalizados:")
df

[1/5] Extraindo dados da API IBGE...
[2/5] Normalizando estrutura JSON...

[3/5] Dados extraídos e normalizados:


,series,id,variavel,unidade,id,nome,categoria.0,categoria.1140,categoria.1141,categoria.1142,...,id,variavel,unidade,localidade.id,localidade.nivel.id,localidade.nivel.nome,localidade.nome,serie.2000,serie.2010,serie.2022
0,"[{'localidade': {'id': '11', 'nivel': {'id': '...",606,População,Pessoas,58,Grupo de idade,Total,NaN,NaN,NaN,...,606,População,Pessoas,11,N3,Unidade da Federação,Rondônia,...,...,1581196
1,"[{'localidade': {'id': '11', 'nivel': {'id': '...",606,População,Pessoas,58,Grupo de idade,NaN,0 a 4 anos,NaN,NaN,...,606,População,Pessoas,12,N3,Unidade da Federação,Acre,...,...,830018
2,"[{'localidade': {'id': '11', 'nivel': {'id': '...",606,População,Pessoas,58,Grupo de idade,NaN,NaN,5 a 9 anos,NaN,...,606,População,Pessoas,13,N3,Unidade da Federação,Amazonas,...,...,3941613
3,"[{'localidade': {'id': '11', 'nivel': {'id': '...",606,População,Pessoas,58,Grupo de idade,NaN,NaN,NaN,10 a 14 anos,...,606,População,Pessoas,14,N3,Unidade da Federação,Roraima,...,...,636707
4,"[{'localidade': {'id': '11', 'nivel': {'id': '...",606,População,Pessoas,58,Grupo de idade,NaN,NaN,NaN,NaN,...,606,População,Pessoas,15,N3,Unidade da Federação,Pará,...,...,8120131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,606,População,Pessoas,43,N3,Unidade da Federação,Rio Grande do Sul,...,...,...
401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,606,População,Pessoas,50,N3,Unidade da Federação,Mato Grosso do Sul,...,...,...
402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,606,População,Pessoas,51,N3,Unidade da Federação,Mato Grosso,...,...,...
403,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,606,População,Pessoas,52,N3,Unidade da Federação,Goiás,...,...,...
